### Task1

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
data = spark.read.csv('Amazon_Responded_Oct05.csv',header = True) # Importing the dataset as a csv

In [4]:
data_rdd = data.rdd # Converting the csv into rdd

### Task1: Step 1

In [5]:
verified_users = data_rdd.filter(lambda x : x['user_verified'] == 'True') 

In [6]:
verified_users.count() # Counting the number of verified users

171797

### Writing a function that gets Month and Day from the column 'tweet_created_at'

In [7]:
def DateStrip(lines):
      lines = lines['tweet_created_at'].split(":")[0][4:10]
      return lines

In [8]:
dates = verified_users.map(DateStrip)

In [9]:
dates.collect()

['Nov 01',
 'Nov 01',
 'Nov 01',
 'Nov 01',
 'Nov 01',
 'Nov 01',
 'Nov 01',
 'Nov 01',
 'Nov 01',
 'Nov 01',
 'Nov 02',
 'Nov 01',
 'Nov 01',
 'Nov 02',
 'Nov 02',
 'Nov 06',
 'Nov 01',
 'Nov 01',
 'Nov 01',
 'Nov 01',
 'Nov 01',
 'Nov 02',
 'Nov 02',
 'Nov 01',
 'Nov 01',
 'Nov 01',
 'Nov 01',
 'Nov 01',
 'Nov 01',
 'Nov 01',
 'Nov 04',
 'Nov 01',
 'Nov 01',
 'Nov 01',
 'Nov 02',
 'Nov 01',
 'Nov 01',
 'Nov 02',
 'Nov 01',
 'Nov 01',
 'Nov 08',
 'Nov 08',
 'Nov 01',
 'Nov 05',
 'Nov 05',
 'Nov 01',
 'Nov 01',
 'Nov 02',
 'Nov 02',
 'Nov 01',
 'Nov 01',
 'Nov 01',
 'Nov 05',
 'Nov 05',
 'Nov 06',
 'Nov 06',
 'Nov 06',
 'Nov 06',
 'Nov 01',
 'Nov 01',
 'Nov 01',
 'Nov 01',
 'Nov 01',
 'Nov 02',
 'Nov 02',
 'Nov 01',
 'Nov 01',
 'Nov 03',
 'Nov 03',
 'Nov 03',
 'Nov 01',
 'Nov 01',
 'Nov 02',
 'Nov 02',
 'Nov 02',
 'Nov 02',
 'Nov 01',
 'Nov 01',
 'Nov 01',
 'Nov 01',
 'Nov 01',
 'Nov 01',
 'Nov 01',
 'Nov 01',
 'Nov 01',
 'Nov 01',
 'Nov 02',
 'Nov 02',
 'Nov 02',
 'Nov 01',
 'Nov 01',

In [10]:
count_dates = dates.countByValue().items()

### Task2 : Step2

In [11]:
count_dates # Jan 03 is the date with maximum number of tweets.

dict_items([('Nov 01', 370), ('Nov 02', 876), ('Nov 06', 357), ('Nov 04', 355), ('Nov 08', 519), ('Nov 05', 1058), ('Nov 03', 440), ('Nov 07', 592), ('Nov 09', 231), ('Nov 12', 720), ('Nov 10', 93), ('Nov 11', 149), ('Nov 13', 226), ('Nov 14', 349), ('Nov 15', 318), ('Nov 16', 104), ('Nov 18', 1246), ('Nov 17', 101), ('Nov 19', 457), ('Nov 20', 789), ('Nov 22', 274), ('Nov 21', 213), ('Nov 23', 907), ('Nov 25', 373), ('Nov 26', 521), ('Nov 24', 593), ('Nov 28', 270), ('Nov 27', 208), ('Nov 29', 952), ('Dec 01', 875), ('Dec 03', 1197), ('Nov 30', 278), ('Dec 04', 884), ('Dec 02', 314), ('Dec 05', 824), ('Dec 07', 360), ('Dec 08', 125), ('Dec 06', 767), ('Dec 09', 431), ('Dec 13', 979), ('Dec 11', 918), ('Dec 12', 372), ('Dec 10', 979), ('Dec 15', 1276), ('Dec 14', 270), ('Dec 17', 221), ('Dec 16', 363), ('Dec 18', 178), ('Dec 19', 240), ('Dec 21', 819), ('Dec 23', 196), ('Dec 20', 150), ('Dec 22', 651), ('Dec 26', 978), ('Dec 24', 247), ('Dec 25', 433), ('Dec 29', 518), ('Dec 27', 442),

In [12]:
# Now taking only those rows in the rdd on which we will be working
new= verified_users.map(lambda row: (row["id_str"], row["tweet_created_at"],row["user_verified"],row["favorite_count"],row["retweet_count"], row["text_"]))


In [13]:
new.first()

("'793281386912354304'",
 'Tue Nov 01 02:39:55 +0000 2016',
 'True',
 '0',
 '0',
 "@SeanEPanjab I'm sorry, we're unable to DM you. Was this order purchased on https://t.co/nUUp5MLhYl, or one of our other sites? ^CL")

### Before performing sum of favorite count and retweet count, we are writing a function that does the following
#### 1) If in both the columns value is present, after typecasting into integer perform sum
#### 2) If either one or both of them has a missing value, then it return -1
#### 3) The addition is susceptible to exception 'Value Error', hence written in try block

In [14]:
def sum(x,y):
    try:
        if(x!= None and y!= None):
            return(int(x)+int(y))
        else:
            return - 1
    except ValueError:
        return -1
            

In [15]:
tweets_100 = new.filter(lambda row : row[1][4:10] == "Jan 03").map(lambda x : (sum(x[3],x[4]),x[5])).sortByKey(False).take(100)


In [16]:
tweets_100 # Getting the top 100 tweets based on the sum count

[(5,
  '@amazon worst shopping  experience,  no service, no substantial reply to complaints, no delivery for 1 week post guarantee date.'),
 (3,
  '@ItsJosshA We always aim to deliver by the date given in your confirmation email. Have we missed that date? Any update in tracking?  ^NF'),
 (2,
  "@ItsJosshA Oh no! I'm sorry! Please reach out to us so that we can look into options: https://t.co/hApLpMlfHN. ^JO"),
 (2,
  '@KStefl Sounds like you know what to add to your Halloween playlist for this year! ^BV'),
 (2, '@Schoey1992 Happy Birthday, Matt! #FriendsForever #FriendshipGoals ^JO'),
 (2, '@ratbones666 You so fancy!!! You already knooow!!! ^EP'),
 (2,
  "@ThorpPerrow Awww! Happy Birthday, you don't look a day over 20 ;-) .Pass on your details here: https://t.co/g52MRb0AOt for lil surprise ^RS"),
 (2,
  "@thedexterouz Hi! We'd like to help. When you have a moment, please connect with us here:  https://t.co/vlvfJr4nN9 ^YP"),
 (1,
  "@matt_linsley Please keep us posted on this and let us

In [19]:
# Converting it in to rdd
rdd_tweets_100 = spark.sparkContext.parallelize(tweets_100)

In [18]:
# Defining th elist of punctuations we will be removing befor doing word count
punctuations = ["(", "[", ",", ".", "!", "?", ":", ";", "]", ")", "@", "^",'#','â€‹hey','ðŸ˜Š','ðŸ˜Œ','ðŸ˜·ðŸ˜·','ðŸ™‹']

### Writing a function to replace punctuations in text with '' "



In [24]:
def eliminate_punctuations(text):
    for char in text:
        if char in punctuations:
          text = text.replace(char,"")
    return text


In [21]:
rdd_tweets_100.first()

(5,
 '@amazon worst shopping  experience,  no service, no substantial reply to complaints, no delivery for 1 week post guarantee date.')

### In word count, we are first removing punctuations, converting into lower text, splitting, and then counting number of occurences
### We are not dealing with  urls, emojis etc

In [22]:
word_count_pair = rdd_tweets_100.map(lambda x : x[1]).map(lambda text : eliminate_punctuations(text)).flatMap(lambda text : text.lower().split()).map(lambda word : (word, 1)).reduceByKey(lambda a,b : a+b).sortByKey(ascending = True).collect()

### Task 1: Step 3

In [23]:
word_count_pair #Sorted on the basis of key i.e. word 

[('&amp', 1),
 ('+', 1),
 ('-', 2),
 ('1', 1),
 ('1/2', 2),
 ('1/3', 2),
 ('2/2', 2),
 ('2/3', 1),
 ('20', 1),
 ('2100', 1),
 ('3-5', 1),
 ('3/3', 1),
 ('3/3sr', 1),
 ('36hrs', 1),
 ('8', 1),
 ('__naijadrew', 1),
 ('_dmarie99', 1),
 ('_izzyscott', 1),
 ('a', 24),
 ('a-to-z', 1),
 ('ab', 1),
 ('abhinavsaroj3', 4),
 ('able', 3),
 ('about', 4),
 ('aboutasix', 1),
 ('above', 2),
 ('access', 3),
 ('account', 2),
 ('accttypically', 1),
 ('add', 1),
 ('address', 1),
 ('adi_baddy', 1),
 ('advise', 1),
 ('ae', 3),
 ('af', 2),
 ('aim', 2),
 ('ak', 1),
 ('alerts', 2),
 ('alexa', 1),
 ('alexisgraham99', 1),
 ('allow', 1),
 ('already', 1),
 ('always', 3),
 ('alyssagoldman', 3),
 ('am', 1),
 ('amazon', 7),
 ('amazonhelp', 1),
 ('amazonin', 2),
 ('ambermullennn', 1),
 ('an', 7),
 ('and', 17),
 ('any', 8),
 ('anything', 1),
 ('apologies', 2),
 ('app', 6),
 ('application', 1),
 ('appreciation', 1),
 ('ar', 3),
 ('are', 2),
 ("aren't", 1),
 ('ariannadelbene', 1),
 ('arrange', 1),
 ('arrive', 6),
 ('arri

In [25]:
import pandas as pd
ans = pd.DataFrame(word_count_pair) # Converting list to Dataframe
ans.columns = (['Word','Frequency'])

In [26]:
ans.to_csv('word_count.csv',index = False)

### Task2

In [28]:
data_2 = spark.read.csv('find_text.csv',header = True) # Loading the csv

In [29]:
data_2_rdd = data_2.rdd # Converting the csv into rdd


In [30]:
rdd_to_join = data_rdd.map(lambda x: (x['id_str'],x['text_'])) # Creating a rdd from original dataset on which we will be performing join

In [31]:
new = data_2_rdd.map(lambda x: (x['id_str'],'1'))# Changing the structure of data_2_rdd as (['id_str'],['1']) for joining
# '1' was appended as dummy otherwise it would throw an exception

In [32]:
final_ans = new.join(rdd_to_join).map(lambda x : (x[0], x[1][1])).collect() # Peforming join

In [33]:
final_ans

[("'793322433625415680'",
  '@Murtazansp Could you please tell us if the product is fulfilled by amazon or fulfilled by a seller? ^NR (2/2)'),
 ("'793382930085253121'",
  "@mybharatraj Hi! Sorry about that. I'd like our team to look into this, please reach out to us via: https://t.co/vlvfJr4nN9 ^SG"),
 ("'793407430344310785'",
  '@AmazonHelp How many times do you want to write back to you guys??? Check the complaints through email regarding  Order 171-1338898-5999507.'),
 ("'793419622011719681'",
  "@amazonIN Hey dudes, when are you gonna take back my returned parcel and give my money back? It's been ages now."),
 ("'793435713089867776'",
  '@AmazonHelp I am so frustrated now! Us customer care also seems helpless. Tell me what to do? https://t.co/0O651QTteH'),
 ("'793443470149087233'",
  "@AmazonVideo I am amazon.de user and yet your android app doesn't recognize me as a German user, and kept asking me for 30 day trail."),
 ("'793445349943803904'",
  "@praveen_pandey_ Hi, I've checked 

In [34]:
import pandas as pd    
df = pd.DataFrame(final_ans) # Converting list to Dataframe

In [35]:
df.columns = (['Id_str','Text'])

In [36]:
df

,Id_str,Text
0,'793322433625415680',@Murtazansp Could you please tell us if the pr...
1,'793382930085253121',@mybharatraj Hi! Sorry about that. I'd like ou...
2,'793407430344310785',@AmazonHelp How many times do you want to writ...
3,'793419622011719681',"@amazonIN Hey dudes, when are you gonna take b..."
4,'793435713089867776',@AmazonHelp I am so frustrated now! Us custome...
...,...,...
53933,'810480535726342144',@AmazonHelp I completed the form via the link ...
53934,'810481606624301056',@AmazonHelp @KindleIndia @amazonIn you have lo...
53935,'810494679770873856',"@anikmankar Hey! As informed earlier, we would..."
53936,'811577323573182464',"""@AmazonHelp It doesn't. I've tried resetting ..."


In [37]:
df.to_csv('find_text_filled.csv',index = False) # Writing the final output to csv